# Fine-tuning and Custom Models

## Overview

This notebook covers fine-tuning techniques for creating custom language models and embeddings. We'll explore:

1. **Fine-tuning Fundamentals**: Understanding when and why to fine-tune
2. **Data Preparation**: Creating high-quality training datasets
3. **Fine-tuning Techniques**: LoRA, QLoRA, and full fine-tuning
4. **Custom Embeddings**: Training domain-specific embedding models
5. **Evaluation and Validation**: Measuring fine-tuned model performance
6. **Production Deployment**: Serving custom models efficiently

## Learning Objectives

By the end of this notebook, you will:
- Understand different fine-tuning approaches and their trade-offs
- Know how to prepare and validate training data
- Be able to fine-tune models using various techniques
- Create custom embedding models for specific domains
- Evaluate and deploy fine-tuned models effectively

---

## Table of Contents

1. [Fine-tuning Fundamentals](#fundamentals)
2. [Data Preparation](#data-prep)
3. [Fine-tuning Techniques](#techniques)
4. [Custom Embeddings](#embeddings)
5. [Evaluation and Validation](#evaluation)
6. [Production Deployment](#deployment)
7. [Real-world Applications](#applications)

In [ ]:
# Install required packages
!pip install -q transformers datasets accelerate peft bitsandbytes torch torchvision torchaudio sentence-transformers scikit-learn matplotlib seaborn wandb

# Import necessary libraries
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from typing import List, Dict, Any, Tuple, Optional
from dataclasses import dataclass
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import (
    AutoTokenizer, AutoModel, AutoModelForCausalLM,
    TrainingArguments, Trainer, DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from datasets import Dataset, load_dataset
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
import bitsandbytes as bnb
from sentence_transformers import SentenceTransformer, InputExample, losses
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# Import our LLM provider utilities
import sys
sys.path.append('../../utils')
from llm_providers import get_available_providers, LLMProviderFactory, LLMConfig

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Initialize LLM provider
available_providers = get_available_providers()
if not available_providers:
    raise ValueError("No LLM providers available! Please check your .env file and API keys.")

provider_name = list(available_providers.keys())[0]
llm_provider = available_providers[provider_name]

print(f"✅ All packages imported successfully!")
print(f"🔧 Using LLM provider: {provider_name.upper()}")
print(f"🌐 Available providers: {list(available_providers.keys())}")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🚀 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print("🔧 Environment configured for fine-tuning and custom models")